In [ ]:
# # Param
# test = "True"

In [ ]:
is_test = True
if test == "False" or test == False:
    is_test = False

In [4]:
dataset = "data"

In [5]:
from utils.data_loader import DataLoader
from utils.data_io import join_path

In [6]:
data_loader = DataLoader(dataset)

In [7]:
train_df = data_loader.get_data_train()
test_df = data_loader.get_data_test()

In [8]:
data_desc = data_loader.get_data_desc()

label_column = data_desc['label_column']
text_column = data_desc['text_column']

In [9]:
keyword_concepts = data_loader.get_keyword_concepts()
keywords = []
for k in keyword_concepts.keys():
    keywords += keyword_concepts[k]

print(keywords)

['coronary', 'myocardial', 'hypertension', 'cardiac', 'systolic', 'colitis', 'esophageal', 'gastrointestinal', 'bowel', 'duodenal', 'defect', 'loss', 'airway', 'graft', 'respiratory', 'cancer', 'carcinoma', 'sarcoma', 'malignancy', 'chemotherapy', 'brain', 'cerebral', 'neuronal', 'motor', 'cord']


In [10]:
abstract_concepts = data_loader.get_abstract_concepts()
abstract_concepts = [ac['abstract_concept_name'] for ac in abstract_concepts]
abstract_concepts

['Cardiac Function and Disorders',
 'Heart Muscle and Blood Pressure',
 'Coronary Artery Issues',
 'Intestinal and Esophageal Conditions',
 'Gastrointestinal Tract Ailments',
 'Inflammatory Bowel Diseases',
 'General Pathological States',
 'Respiratory System Impairments',
 'Tissue and Graft Issues',
 'Malignant Tumors and Growths',
 'Cancer Treatment and Types',
 'Oncological Malignancies',
 'Central and Peripheral Nervous System Disorders',
 'Brain and Cerebral Conditions',
 'Spinal Cord and Motor Function Impairment']

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
le.fit(train_df[label_column])
train_df[label_column] = le.transform(train_df[label_column])
test_df[label_column] = le.transform(test_df[label_column])

In [13]:
labels = list(le.classes_)
labels

['cardiovascular diseases',
 'digestive system diseases',
 'general pathological conditions',
 'neoplasms',
 'nervous system diseases']

In [14]:
if is_test:
    train_df = train_df.groupby(label_column).sample(1)
    test_df = test_df.groupby(label_column).sample(1)

In [15]:
train_texts = train_df[text_column].to_list()
train_labels = train_df[label_column].to_list()
test_texts = test_df[text_column].to_list()
test_labels = test_df[label_column].to_list()

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import networkx as nx
from matplotlib import cm

In [19]:
class ConceptNetwork(nn.Module):
    def __init__(self, concept_names, embedding_dim, keyword_nli_model_path, abstract_nli_model_path):
        super(ConceptNetwork, self).__init__()
        self.embedding_dim = embedding_dim
        self.concept_names = concept_names  # [keywords, abstract_concepts, labels]
        
        # Initialize concept embeddings for each layer
        self.keyword_embeddings = nn.Parameter(torch.randn(len(concept_names[0]), embedding_dim))
        self.abstract_embeddings = nn.Parameter(torch.randn(len(concept_names[1]), embedding_dim))
        self.label_embeddings = nn.Parameter(torch.randn(len(concept_names[2]), embedding_dim))
        
        # Beta parameters for reliability
        self.keyword_betas = nn.Parameter(torch.ones(len(concept_names[0])))
        self.abstract_betas = nn.Parameter(torch.ones(len(concept_names[1])))
        
        # Shared semantic predictor
        self.semantic_predictor = nn.Linear(embedding_dim, 384)  # Adjusted for all-MiniLM-L6-v2 output
        
        # Load NLI scorers
        self.keyword_scorer, self.keyword_tokenizer = load_model(keyword_nli_model_path)
        self.abstract_scorer, self.abstract_tokenizer = load_model(abstract_nli_model_path)
        
        # Final predictor: expects input from abstract layer scores directly
        self.final_predictor = nn.Linear(len(concept_names[1]), len(concept_names[2]))
        
    def forward(self, texts, device):
        batch_size = len(texts)
        
        # Get concept scores for keyword layer
        keyword_scores = []
        for concept_name in self.concept_names[0]:
            inputs = self.keyword_tokenizer(texts, [concept_name] * batch_size, 
                                         return_tensors='pt', padding=True, truncation=True).to(device)
            with torch.no_grad():
                outputs = self.keyword_scorer(**inputs)
                scores = torch.softmax(outputs.logits, dim=-1)[:, 1]  # Probability of label 1
            keyword_scores.append(scores)
        keyword_scores = torch.stack(keyword_scores, dim=1)  # [batch_size, n_keywords]
        
        # Keyword layer output (not used in final prediction, kept for semantic loss)
        keyword_output = keyword_scores @ (self.keyword_embeddings * self.keyword_betas.unsqueeze(-1))
        
        # Abstract layer
        abstract_scores = []
        for concept_name in self.concept_names[1]:
            inputs = self.abstract_tokenizer(texts, [concept_name] * batch_size,
                                          return_tensors='pt', padding=True, truncation=True).to(device)
            with torch.no_grad():
                outputs = self.abstract_scorer(**inputs)
                scores = torch.softmax(outputs.logits, dim=-1)[:, 1]
            abstract_scores.append(scores)
        abstract_scores = torch.stack(abstract_scores, dim=1)  # [batch_size, n_abstract_concepts]
        
        # Label layer: use abstract_scores directly for prediction
        predictions = self.final_predictor(abstract_scores)
        
        # Semantic embeddings for loss computation
        keyword_semantic = self.semantic_predictor(self.keyword_embeddings)
        abstract_semantic = self.semantic_predictor(self.abstract_embeddings)
        label_semantic = self.semantic_predictor(self.label_embeddings)
        
        return predictions, keyword_semantic, abstract_semantic, label_semantic, keyword_scores, abstract_scores

In [ ]:
def train_model(model, train_texts, train_labels, val_texts, val_labels, concept_names, sbert_embeddings,
                batch_size=16, num_epochs=100, patience=5, lambda_semantic=1.0, device='cuda'):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    mse_loss = nn.MSELoss()
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0
        for i in tqdm(range(0, len(train_texts), batch_size), desc=f"Epoch {epoch+1}"):
            batch_texts = train_texts[i:i+batch_size]
            batch_labels = torch.tensor(train_labels[i:i+batch_size], dtype=torch.long).to(device)
            
            optimizer.zero_grad()
            predictions, keyword_semantic, abstract_semantic, label_semantic, _, _ = model(batch_texts, device)
            
            # Predictor loss
            pred_loss = criterion(predictions, batch_labels)
            
            # Semantic loss
            semantic_loss = 0
            for layer_idx, (embeddings, concepts) in enumerate(zip(
                [keyword_semantic, abstract_semantic, label_semantic],
                concept_names)):
                sbert_emb = torch.tensor(sbert_embeddings[layer_idx], device=device, dtype=torch.float32)
                semantic_loss += mse_loss(embeddings, sbert_emb)
            
            total_loss = pred_loss + lambda_semantic * semantic_loss
            total_loss.backward()
            optimizer.step()
            epoch_train_loss += total_loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, len(val_texts), batch_size):
                batch_texts = val_texts[i:i+batch_size]
                batch_labels = torch.tensor(val_labels[i:i+batch_size], dtype=torch.long).to(device)
                
                predictions, keyword_semantic, abstract_semantic, label_semantic, _, _ = model(batch_texts, device)
                pred_loss = criterion(predictions, batch_labels)
                semantic_loss = 0
                for layer_idx, (embeddings, concepts) in enumerate(zip(
                    [keyword_semantic, abstract_semantic, label_semantic],
                    concept_names)):
                    sbert_emb = torch.tensor(sbert_embeddings[layer_idx], device=device, dtype=torch.float32)
                    semantic_loss += mse_loss(embeddings, sbert_emb)
                val_loss += (pred_loss + lambda_semantic * semantic_loss).item()
        
        avg_train_loss = epoch_train_loss / (len(train_texts) // batch_size + 1)
        avg_val_loss = val_loss / (len(val_texts) // batch_size + 1)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break
    
    # Plot losses
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.savefig('loss_plot.png')
    plt.close()
    
    return train_losses, val_losses

In [ ]:
def test_model(model, test_texts, test_labels, concept_names, batch_size=16, device='cuda'):
    model = model.to(device)
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(test_texts), batch_size), desc="Testing"):
            batch_texts = test_texts[i:i+batch_size]
            batch_labels = test_labels[i:i+batch_size]
            
            outputs, _, _, _, _, _ = model(batch_texts, device)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(batch_labels)
    
    print("\nClassification Report:")
    print(classification_report(true_labels, predictions, target_names=concept_names[2]))

In [22]:
def interpretable_prediction(model, test_text, test_label, concept_names, device='cuda'):
    model = model.to(device)
    model.eval()
    
    print("\nInterpretable Prediction for Sample:")
    print(f"Input Text: {test_text}")
    print(f"True Label: {concept_names[2][test_label]}")
    
    with torch.no_grad():
        # Process single sample
        predictions, _, _, _, keyword_scores, abstract_scores = model([test_text], device)
        
        # Keyword layer activations
        print("\nKeyword Layer Activations:")
        for concept_name, score in zip(concept_names[0], keyword_scores[0]):
            print(f"  {concept_name}: {score:.4f}")
        
        # Abstract layer activations
        print("\nAbstract Layer Activations:")
        for concept_name, score in zip(concept_names[1], abstract_scores[0]):
            print(f"  {concept_name}: {score:.4f}")
        
        # Label layer predictions
        label_probs = torch.softmax(predictions, dim=-1)[0]
        print("\nLabel Layer Probabilities:")
        for concept_name, prob in zip(concept_names[2], label_probs):
            print(f"  {concept_name}: {prob:.4f}")
        
        # Final prediction
        predicted_label_idx = torch.argmax(predictions, dim=-1).item()
        print(f"\nPredicted Label: {concept_names[2][predicted_label_idx]}")
        
        return keyword_scores[0].cpu().numpy(), abstract_scores[0].cpu().numpy(), label_probs.cpu().numpy(), predicted_label_idx

In [ ]:
def visualize_network(model, concept_names, keyword_scores, abstract_scores, label_probs, predicted_label_idx, output_file='network_visualization.png'):
    G = nx.DiGraph()
    
    # Add nodes for each layer
    for i, keyword in enumerate(concept_names[0]):
        G.add_node(f"K_{i}", label=keyword, layer='keyword', score=keyword_scores[i])
    
    for i, abstract in enumerate(concept_names[1]):
        G.add_node(f"A_{i}", label=abstract, layer='abstract', score=abstract_scores[i])
    
    for i, label in enumerate(concept_names[2]):
        G.add_node(f"L_{i}", label=label, layer='label', score=label_probs[i])
    
    # Add edges: keyword -> abstract, abstract -> label
    for i in range(len(concept_names[0])):
        for j in range(len(concept_names[1])):
            # Use beta weights for keyword -> abstract edges
            weight = model.keyword_betas[i].item() * model.abstract_betas[j].item()
            G.add_edge(f"K_{i}", f"A_{j}", weight=weight)
    
    for i in range(len(concept_names[1])):
        for j in range(len(concept_names[2])):
            # Use final predictor weights for abstract -> label edges
            weight = model.final_predictor.weight[j, i].item()
            G.add_edge(f"A_{i}", f"L_{j}", weight=weight)
    
    # Define node positions for layered layout
    pos = {}
    max_nodes = max(len(concept_names[0]), len(concept_names[1]), len(concept_names[2]))
    for i, keyword in enumerate(concept_names[0]):
        pos[f"K_{i}"] = (0, max_nodes - i * max_nodes / len(concept_names[0]))
    for i, abstract in enumerate(concept_names[1]):
        pos[f"A_{i}"] = (1, max_nodes - i * max_nodes / len(concept_names[1]))
    for i, label in enumerate(concept_names[2]):
        pos[f"L_{i}"] = (2, max_nodes - i * max_nodes / len(concept_names[2]))
    
    # Node colors and sizes based on scores
    node_colors = []
    node_sizes = []
    for node in G.nodes(data=True):
        score = node[1]['score']
        node_colors.append(score)
        node_sizes.append(500 + score * 2000)  # Scale size based on score
    
    # Edge widths based on weights
    edge_widths = [abs(G[u][v]['weight']) * 2 for u, v in G.edges()]
    
    # Plot the graph
    plt.figure(figsize=(12, 8))
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, cmap=cm.viridis)
    nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.5)
    
    # Add labels (concept names and scores)
    labels = {node: f"{data['label']}\n{data['score']:.2f}" for node, data in G.nodes(data=True)}
    nx.draw_networkx_labels(G, pos, labels, font_size=8)
    
    # Highlight predicted label
    predicted_node = f"L_{predicted_label_idx}"
    nx.draw_networkx_nodes(G, pos, nodelist=[predicted_node], node_color='red', node_size=node_sizes[list(G.nodes).index(predicted_node)])
    
    plt.title("Concept Network Visualization")
    plt.show()
    plt.savefig(output_file)
    plt.close()

In [24]:
def load_model(path):
    model = BertForSequenceClassification.from_pretrained(path)
    tokenizer = BertTokenizer.from_pretrained(path)
    return model, tokenizer

In [25]:
keyword_nli_model_path = join_path(dataset, 'scorer_model', 'keyword_scorer')
abstract_nli_model_path = join_path(dataset, 'scorer_model', 'abstract_scorer')

In [26]:
# SBERT embeddings (precomputed for efficiency)
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
sbert_embeddings = [
    sbert_model.encode(keywords, convert_to_numpy=True),
    sbert_model.encode(abstract_concepts, convert_to_numpy=True),
    sbert_model.encode(labels, convert_to_numpy=True)
]

In [ ]:
concept_names = [keywords, abstract_concepts, labels]
model = ConceptNetwork(
    concept_names, embedding_dim=64, 
    keyword_nli_model_path=keyword_nli_model_path,
    abstract_nli_model_path=abstract_nli_model_path
)

In [28]:
from transformers.utils import logging

logging.set_verbosity_error()

In [ ]:
# Train
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_losses, val_losses = train_model(
    model, train_texts, train_labels, val_texts, val_labels,
    concept_names, sbert_embeddings, batch_size=16, num_epochs=20,
    patience=5, lambda_semantic=0.1, device=device
)

Epoch 1: 100%|██████████| 4/4 [00:35<00:00,  8.76s/it]


Epoch 1, Train Loss: 2.1116, Val Loss: 1.3244


Epoch 2: 100%|██████████| 4/4 [00:38<00:00,  9.57s/it]


Epoch 2, Train Loss: 2.0388, Val Loss: 1.3118


In [31]:
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [ ]:
test_model(model, test_texts, test_labels, concept_names, batch_size=16, device=device)


Classification Report:
                                 precision    recall  f1-score   support

        cardiovascular diseases       0.00      0.00      0.00         1
      digestive system diseases       0.00      0.00      0.00         1
general pathological conditions       0.20      1.00      0.33         1
                      neoplasms       0.00      0.00      0.00         1
        nervous system diseases       0.00      0.00      0.00         1

                       accuracy                           0.20         5
                      macro avg       0.04      0.20      0.07         5
                   weighted avg       0.04      0.20      0.07         5



/Users/dauduchieu/Desktop/iSE-CBM/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/dauduchieu/Desktop/iSE-CBM/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/dauduchieu/Desktop/iSE-CBM/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

In [34]:
# Interpretable prediction and visualization for a single sample
if test_texts and test_labels:
    keyword_scores, abstract_scores, label_probs, predicted_label_idx = interpretable_prediction(
        model, test_texts[0], test_labels[0], concept_names, device=device
    )
    visualize_network(model, concept_names, keyword_scores, abstract_scores, label_probs, predicted_label_idx)


Interpretable Prediction for Sample:
Input Text: Plasma concentrations of epinephrine during CPR in the dog. STUDY OBJECTIVE: The purpose of this study was to evaluate whether the marked increase in the plasma concentrations of epinephrine during cardiopulmonary arrest and basic life support (BLS) could be due in part to decreased distribution and/or elimination. DESIGN AND INTERVENTIONS: Dogs were randomly assigned to undergo adrenalectomy or sham-operation. Some adrenalectomized animals received an epinephrine infusion. MEASUREMENTS AND MAIN RESULTS: In the seven sham-operated dogs, the plasma epinephrine concentrations increased markedly during BLS as expected. In the seven adrenalectomized dogs receiving a constant infusion of epinephrine, cardiopulmonary arrest and BLS induced a three to sixfold increase in plasma epinephrine concentrations, with an increase in the mean plasma epinephrine concentrations (calculated from the area under the curve) of 1.21 +/- 0.12 ng/mL (P less tha